In [ ]:
!pip install --upgrade tensorflow==1.15

Requirement already up-to-date: tensorflow==1.15 in /usr/local/lib/python3.7/dist-packages (1.15.0)


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler #minmax buat bersihin data, Onehot untuk bikin prediksi
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
tf.__version__ 

'1.15.0'

In [ ]:
df = pd.read_csv('winequality-white.csv', sep = ';')
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [ ]:
cols = df.columns
features = df[cols[0:-1]].values #0 itu index start. -1 tuh sampe index -1
label = df[cols[-1]].values.reshape(-1,1)

In [ ]:
#SPLIT DATA
x_train, x_test, y_train, y_test = train_test_split(features, label, test_size=0.2) #buat kasih tau brp test sizenya. split = pisahin

In [ ]:
print(f'Train :{x_train.shape[0]} ') #nunjukin berapa data yang di train

Train :3918 


In [ ]:
#PREPROCESSING FEATURES
scaler = StandardScaler().fit(x_train) #supaya inputnya ada di arrange yang sama/  bisa pake standardscaler, bisa minmax sama aja
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
#PREPROCESSING LABELS
encoder = OneHotEncoder().fit(y_train) #kalo outpunya string, brati pakenya LabelEncoder. krn setiap kelas punya clas sendiri
#encoder.fit(y_train) fit itu buat nge pas2in, bljrin datanya

y_train = encoder.transform(y_train).toarray() #krn encoder returnnya spars matrix. ubah ke array krn mau numpy
y_test = encoder.transform(y_test).toarray()

In [ ]:
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
y_train, y_test = y_train.astype('float32'), y_test.astype('float32')

In [ ]:
epochs = 100
learning_rate = 0.01
n_features = x_train[0].shape[0]
n_classes = y_train[0].shape[0]
n_features

11

In [ ]:
#construct neural network
input_tensor = tf.placeholder(tf.float32, name = 'input')
label_tensor = tf.placeholder(tf.float32, name = 'label')

In [ ]:
neurons = [64, 64, n_classes]
n_layers = len(neurons)

In [ ]:
neurons[2]

7

In [ ]:
parameters = {
    'W1' : tf.Variable(
        tf.random.normal([
            n_features, neurons[0]
        ]),
        tf.float32
    ),
    'B1' : tf.Variable(
        tf.random.normal([1,  neurons[0]]),
        tf.float32
    ),
    'W2' : tf.Variable(
        tf.random.normal([
            neurons[0], neurons[1]
        ]),
        tf.float32
    ),
    'B2' : tf.Variable(
        tf.random.normal([1,  neurons[1]]),
        tf.float32
    ),
    'W3' : tf.Variable(
        tf.random.normal([
            neurons[1], neurons[2]
        ]),
        tf.float32
    ),
    'B3' : tf.Variable(
        tf.random.normal([1,  neurons[2]]),
        tf.float32
    )
}

In [ ]:
#Sama aja kyk yg dibawah
# def forward():
#   z1 = tf.matmul(x, parameters['W1']) + parameters['B1']
#   a1 = tf.nn.tanh(z1)

#   z2 = tf.matmul(x, parameters['W2']) + parameters['B3']
#   a2 = tf.nn.tanh(z2)

#   z3 = tf.matmul(x, parameters['W3']) + parameters['B3']
#   a3 = tf.nn.softmax(z3)
#   return a3

In [ ]:
#FUNCTION FORWARD PASS
def forward_pass(input_tensor,params,n_layer=3): #params itu buat w sm b nya
  a = input_tensor
  for i in range(n_layer):
    W = params[f'W{i+1}']
    B = params[f'B{i+1}']

    z = tf.matmul(a,W) + B #update weight
    if i == n_layer-1:
      a = tf.nn.softmax(z)
    else:
      a = tf.nn.sigmoid(z)

  return a 

In [ ]:
def evaluate_accuracy(y, yhat):
  yield (0.5 * (yhat - y)**2).sum(axis=1).mean()
  yield accuracy_score(np.argmax(y, axis = 1), np.argmax(yhat, axis = 1))

In [ ]:
with tf.Session() as sess:
  logits_tensor = forward_pass(input_tensor, parameters)
  loss_tensor = tf.losses.mean_squared_error(label_tensor, logits_tensor)

  #loss_tensor = tf.reduce_mean(1/2 * (label_tensor - logits_tensor) ** 2) <- klo mau tulis rumus
  #akurasi
  true_preds_tensor = tf.equal(tf.argmax(logits_tensor, axis=1), tf.argmax(label_tensor, axis=1))
  acc_tensor = tf.reduce_mean(tf.cast(true_preds_tensor, tf.float32))

  optimiser = tf.train.AdamOptimizer().minimize(loss_tensor)
  sess.run(tf.global_variables_initializer())
  for i in range(epochs):
    sess.run(
        optimiser,
        feed_dict = {
            input_tensor: x_train,
            label_tensor: y_train
        }
    )

    #Validasi
    if i%10 == 0:
      loss,acc = evaluate_accuracy(y_test,sess.run(
          logits_tensor,
          feed_dict = {
              input_tensor: x_test,
              label_tensor: y_test
          }
      ))

      print(f'loss: {loss:.2f}')
      print(f'Accuracy: {acc * 100}%')

loss: 0.55
Accuracy: 32.6530612244898%
loss: 0.49
Accuracy: 38.6734693877551%
loss: 0.48
Accuracy: 41.224489795918366%
loss: 0.46
Accuracy: 41.734693877551024%
loss: 0.44
Accuracy: 41.83673469387755%
loss: 0.43
Accuracy: 43.16326530612245%
loss: 0.42
Accuracy: 43.36734693877551%
loss: 0.41
Accuracy: 44.285714285714285%
loss: 0.40
Accuracy: 45.51020408163266%
loss: 0.39
Accuracy: 46.326530612244895%
